## Train h1 classifier

We define sub category based on classifier_similarity. And train each subcategory only inside that sub category.
We call the lowest category h1, higher category as h2.

In [ ]:
import os
import sys
import json

import sys


import akagi
print("akagi version:", akagi.__version__)
from akagi.data_sources import RedshiftDataSource, S3DataSource
from akagi.iterator import FileFormat

import numpy as np

import pandas as pd
import glob

In [ ]:
import sys
sys.path.append("../")

In [ ]:
DATA_DIR = ""

BASE_DATA_DIR= os.path.normpath(os.path.join(DATA_DIR, ''))

In [ ]:
# created by classifier_similarity.ipynb
VALID_DISTANCE_DICT_PATH=""

In [ ]:
#Data dirs {train, validation}
TRAIN_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "train"))
VALID_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "valid"))

In [ ]:
BASE_MODEL_DIR = "./trained_model/inceptionv3/"
BASE_NAME = ""
CODE_EXTRACTOR=""

In [ ]:
CATSTORE_PATH = ""

In [ ]:
from models.processor import create_generators

TRAIN_DATAGEN, VALID_DATAGEN = create_generators()


### Downlaod similarity df from S3

In [ ]:
!aws s3 cp 

## Multiple Model Train

In [ ]:
from one_vs_all import create_multiple_model_trainer

def create_multimodeltrainer():
    return create_multiple_model_trainer(base_model_name = BASE_NAME, basedir = BASE_MODEL_DIR, traingen=TRAIN_DATAGEN, validgen=VALID_DATAGEN, extractor_path=CODE_EXTRACTOR)

In [ ]:
multitrainer = create_multimodeltrainer()

### virtual category

In [ ]:
from one_vs_all import setup_filedict_by_vc

In [ ]:
from category import VirtualCategories
vc = VirtualCategories.from_file(CATSTORE_PATH)
catkeys = sorted(list(vc.keys()))
fdict = vc.list_dict(VALID_DATA_DIR)

In [ ]:
setup_filedict_by_vc(multitrainer, vc, TRAIN_DATA_DIR, VALID_DATA_DIR)
multitrainer.sample_valid_files(100, seed=123)

### 178 category

In [ ]:
from modelutils import dir2filedict

In [ ]:
multitrainer.setup_filedict(dir2filedict(TRAIN_DATA_DIR), dir2filedict(VALID_DATA_DIR))
catkeys = sorted(list(multitrainer.train_files_dict.keys()))

### After dict set

In [ ]:
multitrainer.sample_valid_files(100, seed=123)

In [ ]:
distdf = pd.read_pickle(VALID_DISTANCE_DICT_PATH)

In [ ]:
DIST_THRESHOLD = 1.5

In [ ]:
def train_subs(keys):
    for targetkey in keys:
        similarkeyset = set(distdf[targetkey][distdf[targetkey] >= DIST_THRESHOLD].index)
        try:
            multitrainer.train_sub(targetkey, similarkeyset,  eachepochs=5)
        except ValueError as e:
            print("ValueError, skip {0}: {1}".format(targetkey, e))

## Training

In [ ]:
len(catkeys)

In [ ]:
3*159/13, 5*159/13

In [ ]:
catkeys.index('76')

In [ ]:
catkeys.index('33')

In [ ]:
catkeys.index('32')

In [ ]:
notyet = catkeys[94:120]

In [ ]:
len(notyet)

In [ ]:
notyet[6]

In [ ]:
catkeys.index('39')

In [ ]:
nonearest = catkeys[94]
nonearest

In [ ]:
distdf['33'][distdf['33'] >= DIST_THRESHOLD].index

In [ ]:
train_subs(catkeys[94:95])

In [ ]:
train_subs(catkeys[95:100])

In [ ]:
train_subs(catkeys[60:80])

In [ ]:
train_subs(catkeys[80:100])

In [ ]:
train_subs(catkeys[100:120])

In [ ]:
train_subs(catkeys[103:107])

In [ ]:
train_subs(catkeys[0:5])

In [ ]:
train_subs(catkeys[5:65])

In [ ]:
152+17

In [ ]:
train_subs(catkeys[152:169])

### Create code model and put to S3

In [ ]:
from modelutils import dir2filedict

In [ ]:
catkeys =  sorted(list(dir2filedict(VALID_DATA_DIR).keys()))

In [ ]:
targetcats = catkeys[60:94] + catkeys[95:100]

In [ ]:
targetcats = catkeys[103:107]

In [ ]:
DIST_DIR=''


In [ ]:
from codeextractor import best_fullmodel_file_to_codemodel_file_by_basename
import tqdm

In [ ]:
from keras import backend as K

In [ ]:
def convert_with_reset(key):
    K.clear_session()
    best_fullmodel_file_to_codemodel_file_by_basename(BASE_MODEL_DIR, BASE_NAME, key)

In [ ]:
[convert_with_reset("h1_" + key) for key in tqdm.tqdm(targetcats)]

In [ ]:
os.makedirs(DIST_DIR, exist_ok=True)

In [ ]:
from codeextractor import best_code_full_path

In [ ]:
def link_codemodel_to(targetkey):
    codejson, codeweight, _, _ = best_code_full_path(BASE_MODEL_DIR, BASE_NAME, targetkey)
    os.link(codejson, os.path.join(DIST_DIR, os.path.basename(codejson)))
    os.link(codeweight, os.path.join(DIST_DIR, os.path.basename(codeweight)))
    return targetkey

In [ ]:
[link_codemodel_to("h1_"+key) for key in targetcats]

In [ ]:
S3PATH = "" + DIST_DIR

In [ ]:
!aws s3 sync --exclude "*" --include "code*" {DIST_DIR} {S3PATH}  